In [ ]:
# instalar dependências
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://archive.apache.org/dist/spark/spark-3.1.2/spark-3.1.2-bin-hadoop2.7.tgz
!tar xf spark-3.1.2-bin-hadoop2.7.tgz
!pip install pyspark
!pip install -q findspark

**Configurando as variáveis de ambiente**



In [ ]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.1.2-bin-hadoop2.7"

**Montando o drive virtual**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


**Inicializando a plataforma**

In [ ]:
import findspark
findspark.init()

**Iniciando uma sessão**

In [ ]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .master('local[*]') \
    .config("spark.driver.memory", "15g") \
    .appName("Iniciando com Spark") \
    .config('spark.ui.port', '4050') \
    .getOrCreate()

**Criando um Dataframe**

In [ ]:
#HISTORICO COVID 19
path = "/content/drive/MyDrive/ANÁLISE E DESENVOLVIMENTO DE SISTEMAS/4⁰ Período /Bid-Data/Trabalho DATA-TOWN/trabalho_final/database/HIST_PAINEL_COVIDBR_29set2023"
registros_covid_19 = spark.read.csv(path, sep=";", inferSchema=True);

**Mostrando o esquema do Dataframe**

In [ ]:
registros_covid_19.printSchema()

**Mudando o nome das colunas**

In [ ]:
registros_covid_19ColNames = ['regiao', 'estado', 'municipio', 'coduf', 'codmun', 'codRegiaoSaude', 'nomeRegiaoSaude', 'data', 'semanaEpi', 'populacaoTCU2019', 'casosAcumulado', 'casosNovos', 'obitosAcumulado', 'obitosNovos', 'Recuperadosnovos', 'emAcompanhamentoNovos', 'interior/metropolitana']

In [ ]:
for index, colName in enumerate(registros_covid_19ColNames):
  registros_covid_19 = registros_covid_19.withColumnRenamed(f"_c{index}", colName, item = True)
registros_covid_19.limit(3).toPandas()

**Preparando para a mudança  de tipo de dado. De String(texto) para Double(Número com decimal)**

In [ ]:
from pyspark.sql.types import DoubleType, StringType
from pyspark.sql import functions as f

In [ ]:
#Antes de realizar a mudança de tipo é preciso transformar o dado para o padrão compatível.
#O Spark vem com o Double no padrão inglês, onde o
# separador de decimal é um ponto e não uma vírgula
registros_covid_19  = registros_covid_19.withColumn('casosAcumulado', f.regexp_replace('casosAcumulado', ',', '.'))
registros_covid_19 .limit(5).toPandas()

In [ ]:
registros_covid_19  = registros_covid_19.withColumn('casosNovos', f.regexp_replace('casosNovos', ',', '.'))
registros_covid_19 .limit(5).toPandas()

In [ ]:
registros_covid_19  = registros_covid_19.withColumn('data', f.regexp_replace('data', ',', '.'))
registros_covid_19 .limit(5).toPandas()

In [ ]:
registros_covid_19  = registros_covid_19.withColumn('casosNovos', f.regexp_replace('casosNovos', ',', '.'))
registros_covid_19 .limit(5).toPandas()

In [ ]:
registros_covid_19  = registros_covid_19.withColumn('casosNovos', f.regexp_replace('casosNovos', ',', '.'))
registros_covid_19 .limit(5).toPandas()

**Escrevendo o arquivo novo**


In [ ]:
regisitros_covid_19.wrte.csv(
    path='/content/drive/MyDrive/ANÁLISE E DESENVOLVIMENTO DE SISTEMAS/4⁰ Período /Bid-Data/Trabalho DATA-TOWN/trabalho_final/database/HIST_PAINEL_COVIDBR_29set2023', mode='overwrite', sep=';', header=True

)

In [ ]:
registros_covid_19.printSchema()

In [ ]:
registros_covid_19\
  .select('data')\
  .limit(20)\
  .toPandas()


In [ ]:
registros_covid_19\
  .select('*')\
  .filter(registros_covid_19.data.startswith('2022'))\
  .limit(5)\
  .toPandas()

CONSTRUÇÃO DE GRÁFICOS

In [ ]:
import plotly.express as px
c = registros_covid_19.toPandas()
fig = px.histogram(c, x = 'regiao', y = 'casosNovos')
fig.show()

In [ ]:
casosAcumulado.printSchema()

In [ ]:
import plotly.express as px
c = registros_covid_19.toPandas().query()
fig = px.histogram(c, y = "casosNovos", x = "regiao")
fig.show()